In [76]:
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

In [77]:
def lables2id(lables):
    target_id = []
    target_lables = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9']
    for lable in lables:
        target_id.append(target_lables.index(lable))
    return target_id

class CLDataset(Dataset):
  def __init__(self, path, label_path):
    data = pd.read_csv(path)
    if(label_path != None):
      x_data = data.drop(['id'], axis=1).loc[10000, :]
    else:
      x_data = data.drop(['id', 'target'], axis=1)
    if(label_path != None):
      labels_data = pd.read_csv(label_path)
      labels = labels_data.drop(['id'], axis=1)
    else:
      labels = data['target']
    self.x_data = torch.from_numpy(x_data.to_numpy()).float()
    self.y_data = torch.Tensor(lables2id(labels)).long()
    self.len = data.shape[0]

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]
  
  def __len__(self):
    return self.len

In [78]:
train_dataset = CLDataset('../dataset/otto-group-product-classification-challenge/train.csv', None)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = CLDataset('../dataset/otto-group-product-classification-challenge/test.csv', '../dataset/otto-group-product-classification-challenge/sampleSubmission.csv')
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [79]:
train_dataset.x_data.shape,test_loader.dataset.x_data.shape

(torch.Size([61878, 93]), torch.Size([93]))

In [80]:
class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.linear1 = torch.nn.Linear(93, 64)
    self.linear2 = torch.nn.Linear(64, 32)
    self.linear3 = torch.nn.Linear(32, 16)
    self.linear4 = torch.nn.Linear(16, 9)
    self.relu = torch.nn.ReLU()

  def forward(self, x):
    x = self.relu(self.linear1(x))
    x = self.relu(self.linear2(x))
    x = self.relu(self.linear3(x))
    x = self.linear4(x)
    return x
   
model = Net()

In [81]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
  running_loss = 0.0
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if batch_idx % 100 == 99:
      print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 100))
      running_loss = 0.0
    

In [82]:
def test():
  correct = 0
  total = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = model(data)
      _, predicted = torch.max(output.data, 1)
      total += target.size(0)
      correct += (predicted == target).sum().item()
  
  print('Accuracy of the network on the test data: %d %%' % (100 * correct / total))

In [84]:
for epoch in range(10):
  train(epoch)
  # test()

[1,   100] loss: 0.778
[1,   200] loss: 0.770
[1,   300] loss: 0.747
[1,   400] loss: 0.745
[1,   500] loss: 0.739
[1,   600] loss: 0.712
[1,   700] loss: 0.710
[1,   800] loss: 0.716
[1,   900] loss: 0.683
[2,   100] loss: 0.691
[2,   200] loss: 0.697
[2,   300] loss: 0.667
[2,   400] loss: 0.680
[2,   500] loss: 0.680
[2,   600] loss: 0.673
[2,   700] loss: 0.691
[2,   800] loss: 0.656
[2,   900] loss: 0.665
[3,   100] loss: 0.636
[3,   200] loss: 0.664
[3,   300] loss: 0.657
[3,   400] loss: 0.627
[3,   500] loss: 0.650
[3,   600] loss: 0.649
[3,   700] loss: 0.643
[3,   800] loss: 0.635
[3,   900] loss: 0.652
[4,   100] loss: 0.626
[4,   200] loss: 0.637
[4,   300] loss: 0.632
[4,   400] loss: 0.624
[4,   500] loss: 0.622
[4,   600] loss: 0.630
[4,   700] loss: 0.604
[4,   800] loss: 0.624
[4,   900] loss: 0.601
[5,   100] loss: 0.594
[5,   200] loss: 0.618
[5,   300] loss: 0.599
[5,   400] loss: 0.612
[5,   500] loss: 0.591
[5,   600] loss: 0.603
[5,   700] loss: 0.610
[5,   800] 